In [29]:
import numpy as np
datos = np.load('tsp.md.npy')

In [30]:
import random
import time
import matplotlib.pyplot as plt
# Número de ciudades
n = len(datos)  # Ajusta el número de ciudades según tu problema
# Parmetros del algoritmo genético
Sc = list(range(1, len(datos)+1))
tamano_poblacion = 100
tiempo_limite = 3600
tiempo_inicio = time.time()

indx = 0
num_generaciones = 0

longitud_cromosoma= int(np.round(np.log2(float(np.math.factorial(n-1)))))

# Inicializar la población



# Función para generar una población inicial
def generar_poblacion(tamano_poblacion, longitud_cromosoma):
    return [''.join(random.choice('01') for _ in range(longitud_cromosoma)) for _ in range(tamano_poblacion)]

# Función para evaluar la aptitud de un individuo
def evaluar_aptitud(cromosoma):
    # Paso 1: Convertir el cromosoma en un número decimal v
    v = int(cromosoma, 2)
    # Paso 2: Asignar a c el número de ciudades menos 1
    c = n
    # Inicializar el vector S
    S = []
    # Evaluar la aptitud
    ias=0
    while c > 0:
        # Paso 3: Hallar el resto de la división entre v y c y añadírselo al vector S
        resto = v % c
        S.append(resto)
        # Paso 4: Asignarle a v el resultado entero de la división entre v y c
        v = v // c
        # Paso 5: Restarle 1 a c
        c -= 1
    # Paso 6: Si c es mayor que 1, volver al Paso 3
    Sn = []
    Scaux = Sc.copy()
    for posicion in S:
        valor_seleccionado = Scaux.pop(posicion)
        Sn.append(valor_seleccionado)
    for ciudad in Scaux:
        if not ciudad in Sn:
            Sn.append(ciudad)

    aptitud = calcular_distancia_total(Sn)

    return aptitud

# Función para calcular la distancia total recorrida (por ejemplo)
def calcular_distancia_total(individuo):
    # Aquí implementa el cálculo de la distancia total basado en el vector S
        ciudad_actual = individuo[0]  # Obtén la primera ciudad
        distancia_total = 0

        for i in range(1, len(datos)):
            siguiente_ciudad = individuo[i]

            distancia_total += datos[ciudad_actual-1][siguiente_ciudad-1]
            ciudad_actual = siguiente_ciudad

        # Regresa a la ciudad de inicio
        distancia_total += datos[ciudad_actual-1][individuo[0]-1]

        return distancia_total




def seleccionar_individuo_ruleta(poblacion, aptitudes):
    total_aptitud = sum(aptitudes)
    probabilidad = [aptitud / total_aptitud for aptitud in aptitudes]
    return random.choices(poblacion, probabilidad)[0]


def seleccionar_individuo_torneo(poblacion, aptitudes):
    k=20
    population_size = len(poblacion)
    tournament = random.sample(range(population_size), k)  # Selecciona 'k' individuos al azar para el torneo
    winner = min(tournament, key=lambda i: aptitudes[i])  # Elige al individuo con la menor aptitud en el torneo
    winner = poblacion[winner]
    return winner



def find_best_individual(population, fitness_values):
    # Encontrar y registrar al mejor individuo en la población.
    best_index = fitness_values.index(min(fitness_values))
    best_individual = population[best_index]

    return best_individual

def replace_worst_with_best(population, fitness_values, best_individual):
    # Encontrar el peor individuo y sustituirlo por el mejor individuo si es superior.
    worst_index = fitness_values.index(min(fitness_values))
    if evaluar_aptitud(best_individual) > fitness_values[worst_index]:
        population[worst_index] = best_individual

def cruzamiento_un_punto(padre1, padre2):
    punto_cruzamiento = random.randint(1, len(padre1)-1)  # Se elige un punto de cruzamiento aleatorio (excepto el primer y último bit)
   
    hijo1 = padre1[:punto_cruzamiento] + padre2[punto_cruzamiento:]
    hijo2 = padre2[:punto_cruzamiento] + padre1[punto_cruzamiento:]

    return hijo1, hijo2







# Función para aplicar la mutación a un individuo
def mutar_individuo(individuo, num_generaciones):
    probabilidad_mutacion = 0.1
    if num_generaciones > 500000:
        probabilidad_mutacion = probabilidad_mutacion/2
    if num_generaciones > 1000000:
        probabilidad_mutacion = probabilidad_mutacion/4
    if num_generaciones > 150000:
        probabilidad_mutacion = probabilidad_mutacion/8
    mutado = list(individuo)
    for i in range(len(individuo)):
        if random.random() < probabilidad_mutacion:
            # Cambia el bit si la probabilidad de mutación se cumple
            mutado[i] = '0' if individuo[i] == '1' else '1'
    return ''.join(mutado)

# Aplicar mutación a la población después del cruzamiento

def convmejor(cromosoma):
   # Paso 1: Convertir el cromosoma en un número decimal v
    v = int(cromosoma, 2)

    # Paso 2: Asignar a c el número de ciudades menos 1
    c = n

    # Inicializar el vector S
    S = []

    # Evaluar la aptitud
    ias=0
    while c > 0:
        # Paso 3: Hallar el resto de la división entre v y c y añadírselo al vector S
        resto = v % c
        S.append(resto)

        # Paso 4: Asignarle a v el resultado entero de la división entre v y c
        v = v // c

        # Paso 5: Restarle 1 a c
        c -= 1

    # Paso 6: Si c es mayor que 1, volver al Paso 3
    Sn = []
    Scaux = Sc.copy()

    for posicion in S:

        valor_seleccionado = Scaux.pop(posicion)
        Sn.append(valor_seleccionado)

    for ciudad in Scaux:
        if not ciudad in Sn:
            Sn.append(ciudad)


    return Sn
poblacion = generar_poblacion(tamano_poblacion, longitud_cromosoma)
aptitudgraph = []
# Algoritmo genético
while indx < 1:
    # Evaluación de aptitud
    aptitudes = [evaluar_aptitud(cromosoma) for cromosoma in poblacion]
    mejor_individuo = find_best_individual(poblacion, aptitudes)
    # Selección
    nueva_poblacion = []


    for _ in range(tamano_poblacion // 2):
        padre1 = seleccionar_individuo_ruleta(poblacion, aptitudes)
        padre2 = seleccionar_individuo_ruleta(poblacion, aptitudes)

        hijo1, hijo2 = cruzamiento_un_punto(padre1, padre2)
        nueva_poblacion.extend([hijo1, hijo2])

    #for _ in range(tamano_poblacion // 2):
        #selected_parents = tournament_selection(poblacion, aptitudes)
        #hijo1, hijo2 = cruzamiento_un_punto(selected_parents[0], selected_parents[1])
        #nueva_poblacion.extend([hijo1, hijo2])
    # Mutación
    nueva_poblacion = [mutar_individuo(individuo,num_generaciones) for individuo in nueva_poblacion]

    # Reemplazar la población anterior con la nueva población
    poblacion = nueva_poblacion
    poblacion[aptitudes.index(max(aptitudes))] = mejor_individuo
    #replace_worst_with_best(poblacion, aptitudes, mejor_individuo)

    # Mostrar la aptitud del mejor individuo en esta generación
    aptitudes = [evaluar_aptitud(cromosoma) for cromosoma in poblacion]
    mejor_individuo_generacion = poblacion[aptitudes.index(min(aptitudes))]
    mejor_aptitud_generacion = min(aptitudes)
    num_generaciones += 1
    if(num_generaciones % 1 == 0): # Every 1 epochs


        print(f"Generación {num_generaciones}: Mejor individuo - {mejor_individuo_generacion}, Aptitud: {mejor_aptitud_generacion}")
    
    aptitudgraph.append(mejor_aptitud_generacion)
    num_generaciones += 1
    tiempo_transcurrido = time.time() - tiempo_inicio
    if tiempo_transcurrido >= tiempo_limite: indx += 1
print(f"mejor solucion Aptitud: {mejor_aptitud_generacion}")
iteraciones = list(range(1, len(aptitudgraph) + 1))

# Crea la gráfica de línea
plt.figure(figsize=(8, 6))
plt.plot(iteraciones, aptitudgraph, marker='o', linestyle='-')
plt.title('Actualización de la Aptitud a lo largo del Tiempo')
plt.xlabel('Iteraciones o Tiempo')
plt.ylabel('Aptitud')
plt.grid(True)

# Muestra la gráfica
plt.show()


Generación 1: Mejor individuo - 01101011100010101111010100001100001000011100011010000001010101100010110010100010101001100011101101101100100001010101000011110101001111111101011100100110110000011010000011010010010110011101011010111011000001001001100110011110110000100000101000000001010100100000000010111010100100001011001101101110011010011100111001010110110100010111011111010011101011010010101000010001001001010001001101111111001011100010111001010111111011001001111011000011111011010001101110111100110101011000010111001110111000111010010100111110101011, Aptitud: 70338.07
Generación 3: Mejor individuo - 01101011100010101111010100001100001000011100011010000001010101100010110010100010101001100011101101101100100001010101000011110101001111111101011100100110110000011010000011010010010110011101011010111011000001001001100110011110110000100000101000000001010100100000000010111010100100001011001101101110011010011100111001010110110100010111011111010011101011010010101000010001001001010001001101111111001011

KeyboardInterrupt: 